In [1]:
import wbgapi as wb
import wbdata
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import r2_score

import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')

<h1 style="color:black; background-color:white; padding:10px; padding-bottom:10px;text-align: center;">Green dimensions</h1>

---